# iML1515修改

In [1]:
import cobra 
from cobra import Model, Reaction, Metabolite
import pandas as pd   

In [8]:
gpr_df = pd.read_excel('data/iML1515-modify_gpr.xlsx',index_col='id')
change_direction_df = pd.read_excel('data/iML1515_direction_change-58.xlsx',index_col='id')
model = cobra.io.read_sbml_model('data/iML1515.xml')

## 1、修改错误GPR关系

In [3]:
change_rxn_gpr_list = list(gpr_df.index[0:14])
print(len(change_rxn_gpr_list))

for r in model.reactions:
    if r.id in change_rxn_gpr_list:
        r.gene_reaction_rule = gpr_df.loc[r.id]['Modified genes ']
        print(r,r.gene_reaction_rule)

14
FMNRx2: fmn_c + h_c + nadph_c --> fmnh2_c + nadp_c b0937 or b3844 or b2764
ARGabcpp: arg__L_p + atp_c + h2o_c --> adp_c + arg__L_c + h_c + pi_c ( b2310 and b2308 and b2306 and b2307 ) or ( b0862 and b0860 and b0864 and b0861 ) or ( b0862 and b0864 and b0863 and b0861 )
ECAP1pp: 2.0 unagamuf_p --> eca2und_p + h_p + udcpdp_p b3785 and b3793 and b3792
ECAP2pp: eca2und_p + unagamuf_p --> eca3und_p + h_p + udcpdp_p b3785 and b3793 and b3792
3NTD4pp: 3cmp_p + h2o_p --> cytd_p + pi_p b0383 or b4213 or b2744
GLUDy: glu__L_c + h2o_c + nadp_c <=> akg_c + h_c + nadph_c + nh4_c b1761
FADRx2: fad_c + h_c + nadph_c --> fadh2_c + nadp_c b2764
ECAP3pp: eca3und_p + unagamuf_p --> eca4und_p + h_p + udcpdp_p b3785 and b3793 and b3792
BWCOGDS1: bwco_c + gtp_c + h_c --> bwco1gdp_c + ppi_c b3857
BWCOGDS2: bwco1gdp_c + gtp_c + h_c --> bwcogdp_c + ppi_c b3857
THZPSN3: atp_c + dhgly_c + dxyl5p_c + h_c + iscssh_c + nadph_c --> 4mpetz_c + amp_c + co2_c + 2.0 h2o_c + iscs_c + nadp_c + ppi_c b3992 and b3990 and

## 2、修改反应方向

In [4]:
for r in model.reactions:
    for i in change_direction_df.index:
        if r.id==i:
            if change_direction_df.loc[i]['改后']=='正向':
                r.bounds=(0,1000)
            if change_direction_df.loc[i]['改后']=='反向':
                r.bounds=(-1000,0)

# 3、反应拆分
SHK3Dr 3dhsk_c + h_c + nadph_c <=> nadp_c + skm_c  
不同的方向由不同的酶催化 ecocyc SHIKIMATE-5-DEHYDROGENASE-RXN

In [5]:
model.reactions.get_by_id('SHK3Dr').bounds = (0,1000)
model.reactions.get_by_id('SHK3Dr').id = 'SHK3Dr_forward'
model.reactions.get_by_id('SHK3Dr_forward').gene_reaction_rule = 'b3281'

reaction = Reaction('SHK3Dr_reverse')
model.add_reaction(reaction)
reaction.build_reaction_from_string('3dhsk_c + h_c + nadph_c <-- nadp_c + skm_c')
model.reactions.get_by_id('SHK3Dr_reverse').gene_reaction_rule = 'b1692'

In [6]:
cobra.io.write_sbml_model(model, "result/iML1515-2020922.xml")

In [7]:
cobra.io.save_json_model(model, "result/iML151-2020922.json")